In [1]:
import numpy as np
print(np.__version__)

1.26.4


In [2]:
# %pip install scikit-surprise
# %pip install sentence-transformers

import pandas as pd
import numpy as np
from numpy import random
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from sentence_transformers import SentenceTransformer


c:\Users\Jack\AI_ML_Course\brandmatch_proj\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [28]:
# Check if result are good?
# pd.set_option('display.max_colwidth', None) 
# recommended_id = [
#   "c7e1249ffc03eb9ded908c236bd1996d",
#   "57aeee35c98205091e18d1140e9f38cf",
#   "e96ed478dab8595a7dbda4cbcbee168f",
#   "4c56ff4ce4aaf9573aa5dff913df997a"
# ]
recommended_id = [
  "c0c7c76d30bd3dcaefc96f40275bdc0a",
  "00411460f7c92d2124a67ea0f4cb5f85",
  "a01a0380ca3c61428c26a231f0e49a09",
  "06409663226af2f3114485aa4e0a23b4"
]

filtered_df = df[df['brand_id'].isin(recommended_id)]
# print(filtered_df.head()['review_content'].to_string())
filter_deep = filtered_df.drop_duplicates(subset='brand_id', keep='first')
filter_deep
# print(filter_deep['review_content'])

,brand_id,review_id,avg_rating,is_product_recommended,review_content,is_anonymous,display_name,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,sentiment_score
0,00411460f7c92d2124a67ea0f4cb5f85,2598,5.0,True,basically stick peanut butter one flavor theyre terrible peanut butter one bestseller big bag small bag doesnt matter one sold took little sell peanut butter cup sometimes would get two case every week go fast seller would love get promo price customer complaint sometimes big bag maybe little bit pricey sell 899 maybe cost price could go little bit would appealing,False,Snacks,5.0,5.0,5.0,5.0,5.0,5.0,1
103,06409663226af2f3114485aa4e0a23b4,3425,5.0,True,overall sell well experience slowdown summertime business tends slower season,False,Snacks,5.0,5.0,5.0,5.0,5.0,5.0,3
1340,a01a0380ca3c61428c26a231f0e49a09,2608,4.0,True,wish could say selling better honest ive tried flavor like apple cinnamon lot think thats probably best seller think coffee particular dont really see move much one selling seems kind,False,Snacks,4.0,4.0,4.0,4.0,4.0,4.0,2
1620,c0c7c76d30bd3dcaefc96f40275bdc0a,2449,4.0,True,shelf taker explaining product different traditional one ni think promotion needed well theyve one thing hasnt sale since product,False,Snacks,4.0,4.0,4.0,4.0,4.0,4.0,1


In [3]:

# Load pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [4]:
df = pd.read_csv("../data/cleaned_df.csv")

In [5]:
df.head()

,brand_id,review_id,avg_rating,is_product_recommended,review_content,is_anonymous,display_name,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,sentiment_score
0,00411460f7c92d2124a67ea0f4cb5f85,2598,5.0,True,basically stick peanut butter one flavor theyr...,False,Snacks,5.0,5.0,5.0,5.0,5.0,5.0,1
1,00411460f7c92d2124a67ea0f4cb5f85,3718,5.0,True,carry currently stock selling well much need p...,True,Snacks,5.0,5.0,5.0,5.0,5.0,5.0,5
2,00411460f7c92d2124a67ea0f4cb5f85,3719,4.0,True,carry whim chocolate havent ordered recently m...,True,Snacks,4.0,4.0,4.0,4.0,4.0,4.0,3
3,00411460f7c92d2124a67ea0f4cb5f85,2004,5.0,True,good flavor selling fast tried time customer s...,False,Snacks,5.0,5.0,5.0,5.0,5.0,5.0,4
4,00411460f7c92d2124a67ea0f4cb5f85,3717,4.0,True,still carry whim selling well among flavor pea...,True,Snacks,4.0,4.0,4.0,4.0,4.0,4.0,5


In [6]:
test_list = [
  "c0c7c76d30bd3dcaefc96f40275bdc0a",
  "66808e327dc79d135ba18e051673d906",
  "00411460f7c92d2124a67ea0f4cb5f85",
  "4f6ffe13a5d75b2d6a3923922b3922e5",
  "e96ed478dab8595a7dbda4cbcbee168f"
]

for brand_to_search in test_list:
  review = df.loc[df['brand_id'] == brand_to_search, 'review_content']
  if not review.empty:
    print(f"Review for brand_id {brand_to_search}: {review.iloc[0]}")
  else:
    print(f"No review found for brand_id {brand_to_search}.")

Review for brand_id c0c7c76d30bd3dcaefc96f40275bdc0a: shelf taker explaining product different traditional one ni think promotion needed well theyve one thing hasnt sale since product
Review for brand_id 66808e327dc79d135ba18e051673d906: one better seller peanut butter flavor popular order distributor well probably keep carrying havent issue know little confusing first changed packaging customer gotten used flavor always good thing also like going unfi im probably going use shipper anymore always problem
Review for brand_id 00411460f7c92d2124a67ea0f4cb5f85: basically stick peanut butter one flavor theyre terrible peanut butter one bestseller big bag small bag doesnt matter one sold took little sell peanut butter cup sometimes would get two case every week go fast seller would love get promo price customer complaint sometimes big bag maybe little bit pricey sell 899 maybe cost price could go little bit would appealing
Review for brand_id 4f6ffe13a5d75b2d6a3923922b3922e5: recently sale s

In [7]:
fake_user_inp = df.iloc[0]
print(fake_user_inp)

# Generate embedding for the fake user input
fake_user_embedding = model.encode(fake_user_inp['review_content'])

brand_id                                   00411460f7c92d2124a67ea0f4cb5f85
review_id                                                              2598
avg_rating                                                              5.0
is_product_recommended                                                 True
review_content            basically stick peanut butter one flavor theyr...
is_anonymous                                                          False
display_name                                                         Snacks
rating_packaging                                                        5.0
rating_price                                                            5.0
rating_quality                                                          5.0
rating_service                                                          5.0
rating_shipping                                                         5.0
rating_taste                                                            5.0
sentiment_sc

In [8]:

df_copy = df.copy()
# Handle missing values in 'review_content' by filling with an empty string
df_copy['review_content'] = df_copy['review_content'].fillna('')

# This will return hybrid_similarity matrix
df_copy['review_embedding'] = df_copy['review_content'].apply(lambda x: model.encode(x))

# Prepare ratings matrix for SVD (pivot table)
ratings_matrix = df_copy.pivot_table(index='review_id', columns='brand_id', values='avg_rating').fillna(0)

# Apply SVD on the ratings matrix
svd = TruncatedSVD(n_components=5)
latent_matrix = svd.fit_transform(ratings_matrix)

# Cosine similarity on review embeddings
embeddings = np.vstack(df_copy['review_embedding'].to_numpy())
embedding_similarity = cosine_similarity(embeddings)
# Combine SVD and content similarity scores
hybrid_similarity = 0.5 * cosine_similarity(latent_matrix) + 0.5 * embedding_similarity
hybrid_similarity


array([[0.99999994, 0.69182222, 0.57197027, ..., 0.63012702, 0.44221117,
        0.6816302 ],
       [0.69182222, 0.99999982, 0.63064843, ..., 0.60075488, 0.42163411,
        0.65368522],
       [0.57197027, 0.63064843, 1.        , ..., 0.4814732 , 0.61457598,
        0.45838851],
       ...,
       [0.63012702, 0.60075488, 0.4814732 , ..., 0.99999991, 0.26457074,
        0.74868256],
       [0.44221117, 0.42163411, 0.61457598, ..., 0.26457074, 1.        ,
        0.3482466 ],
       [0.6816302 , 0.65368522, 0.45838851, ..., 0.74868256, 0.3482466 ,
        0.99999997]])

In [26]:
# Generate embedding for the fake user input
fake_user_embedding = model.encode(fake_user_inp['review_content'])

# Calculate similarity scores between the fake user input and the reviews in the DataFrame
fake_user_embedding = fake_user_embedding.reshape(1, -1)
embedding_similarity = cosine_similarity(fake_user_embedding, embeddings).flatten()

# Assuming hybrid_similarity is already calculated
latent_matrix_similarity = hybrid_similarity.mean(axis=0)

combined_similarity = 0.5 * latent_matrix_similarity + 0.5 * embedding_similarity

In [31]:
# Generate recommendations based on the similarity scores
def recommend_brands_based_on_input(similarity_scores, df, num_recommendations=5):
    similar_reviews = list(enumerate(similarity_scores))
    similar_reviews = sorted(similar_reviews, key=lambda x: x[1], reverse=True)
    
    recommended_brands = set()
    selected_rows = []

    for i in similar_reviews:
        brand_id = df.iloc[i[0]]['brand_id']
        if brand_id not in recommended_brands:
            recommended_brands.add(brand_id)
            selected_rows.append(i[0])
        if len(recommended_brands) == num_recommendations:
            break

    filtered_df = df.iloc[selected_rows]
    return filtered_df

# Get recommendations
top_brands_df = recommend_brands_based_on_input(combined_similarity, df, num_recommendations=40)
top_brands_df.head()

,brand_id,review_id,avg_rating,is_product_recommended,review_content,is_anonymous,display_name,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,sentiment_score
0,00411460f7c92d2124a67ea0f4cb5f85,2598,5.0,True,basically stick peanut butter one flavor theyr...,False,Snacks,5.0,5.0,5.0,5.0,5.0,5.0,1
1606,beed13602b9b0e6ecb5b568ff5058f07,1979,5.0,True,people really like know order couple case ever...,False,Non-Alcoholic Beverage,5.0,5.0,5.0,5.0,5.0,5.0,4
1251,9a1158154dfa42caddbd0694a4e9bdc8,3339,4.0,True,would say hit like lot eat probably half inven...,False,Snacks,4.0,4.0,4.0,4.0,4.0,4.0,1
882,5ef698cd9fe650923ea331c15af3b160,1630,5.0,True,sell also dont carry ton see customer buying q...,False,Powder & Mixes,5.0,5.0,5.0,5.0,5.0,5.0,3
1002,7a614fd06c325499f1680b9896beedeb,2155,3.0,True,actually one slowest seller lot trouble moving...,False,Non-Alcoholic Beverage,3.0,3.0,3.0,3.0,3.0,3.0,1


In [32]:
# def recommend_brands(review_index, top_n=10):
#     # Get similarity scores for the given review
#     similar_reviews = list(enumerate(hybrid_similarity[review_index]))
#     print(similar_reviews)
#     # Sort by similarity and select top reviews (excluding the review itself)
#     similar_reviews = sorted(similar_reviews, key=lambda x: x[1], reverse=True)
#     print(similar_reviews)
#     recommended_brands = set()
#     selected_rows = []

#     for i in similar_reviews[1:]:  # Start from index 1 to skip the review itself
#         brand_id = df.iloc[i[0]]['brand_id']

#         # Add only if brand_id is not yet in recommended_brands
#         if brand_id not in recommended_brands:
#             recommended_brands.add(brand_id)
#             selected_rows.append(i[0])  # Store the index of this row

#         # Stop if we've collected enough unique brand IDs
#         if len(recommended_brands) == top_n:
#             break

#     # Filter the DataFrame to only include the selected rows
#     filtered_df = df.iloc[selected_rows]

#     return filtered_df

# # Example: Recommend top 10 brands for the first review (index 0)
# top_brands_df = recommend_brands(review_index=0, top_n=40)
# top_brands_df.head()


In [33]:
top_brands_df.isnull().sum()

brand_id                  0
review_id                 0
avg_rating                0
is_product_recommended    0
review_content            0
is_anonymous              1
display_name              0
rating_packaging          0
rating_price              0
rating_quality            0
rating_service            0
rating_shipping           0
rating_taste              0
sentiment_score           0
dtype: int64

In [34]:
num_unique_ids = top_brands_df['brand_id'].nunique()
print(num_unique_ids)

40


# Start Filtering

## Normalize the ratings types

In [12]:
category = "Snacks"
rating_columns = ["rating_packaging", "rating_price", "rating_quality", "rating_service", "rating_shipping", "rating_taste", "avg_rating", "sentiment_score"]
scaler = MinMaxScaler()
df_norm = top_brands_df.copy()
df_norm = df_norm[df_norm["display_name"] == "Snacks"]
df_norm[rating_columns] = scaler.fit_transform(df_norm[rating_columns])

In [13]:
num_unique_ids = df_norm['brand_id'].nunique()
print(num_unique_ids)

10


## Assign weights to rating types (Hard Code for now)

In [14]:
random_weights = random.rand(7)
random_weights

array([0.70682274, 0.60170271, 0.77066022, 0.96678918, 0.96349508,
       0.27436838, 0.74358268])

In [15]:
weights_map = {
    "rating_packaging": random_weights[0],
    "rating_price": random_weights[1],
    "rating_quality": random_weights[2],
    "rating_service": random_weights[3],
    "rating_shipping": random_weights[4],
    "rating_taste": random_weights[5],
    "avg_rating": random_weights[6],
    "sentiment_score": 1.0  # Highest weight assigned to sentiment_score
}

# NOTE: We would love to handle case where user does not input anything
# if len(inp_weight) = 0:
#     sort by average_rating
# if some specific rating is not filled
#     we change the weights_map

In [16]:
rating_priorities = []
weights = []
for key in weights_map:
    rating_priorities.append(key)
    weights.append(weights_map[key])

print(rating_priorities)
print(weights)

['rating_packaging', 'rating_price', 'rating_quality', 'rating_service', 'rating_shipping', 'rating_taste', 'avg_rating', 'sentiment_score']
[0.7068227433213164, 0.601702711840327, 0.7706602230948462, 0.9667891845149716, 0.9634950812336437, 0.27436838077870207, 0.7435826843441546, 1.0]


In [17]:
#Apply weight to the rating type in df
weighted_df = df_norm.copy()
weighted_df[rating_priorities] = df_norm[rating_priorities].apply(lambda x: x * weights, axis=1)

In [18]:
weighted_df

,brand_id,review_id,avg_rating,is_product_recommended,review_content,is_anonymous,display_name,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,sentiment_score,review_embedding
1502,bbcbff5c1f1ded46c25d28119a85c6c2,3152,0.371791,True,popular hence sold people love often buy bulk ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,1.00,"[-0.21371414, 0.06628278, -0.023210816, -0.421..."
954,69cb3ea317a32c4e6143e665fdb20b14,2782,0.371791,True,people love taste packaging really cool funky ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.75,"[-0.3612504, -0.35247555, -0.15716112, -0.0831..."
1956,f8c1f23d6a8d8d7904fc0ea8e066b3bb,3171,0.000000,True,think sale werent high enough decided start se...,True,Snacks,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,"[0.031356554, -0.36868742, -0.1400484, -0.2579..."
1658,c7e1249ffc03eb9ded908c236bd1996d,2962,0.743583,True,selling moving pretty well first got much thin...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.50,"[-0.30708426, -0.73055637, -0.2741928, -0.2398..."
1344,a01a0380ca3c61428c26a231f0e49a09,1922,0.371791,True,label price fine last 30 day weve sold twelve ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.75,"[-0.42296505, 0.0075865746, -0.20753312, 0.198..."
719,4f6ffe13a5d75b2d6a3923922b3922e5,2924,0.743583,True,like sell reorder everything good since still ...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,1.00,"[-0.31779307, -0.18992049, -0.039039858, -0.22..."
1643,c3e878e27f52e2a57ace4d9a76fd9acf,1877,0.371791,True,great expired frozen popsicle stick think pric...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,1.00,"[-0.5850852, 0.108858824, -0.07353058, -0.1312..."
1,00411460f7c92d2124a67ea0f4cb5f85,3718,0.743583,True,carry currently stock selling well much need p...,True,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,1.00,"[-0.5571632, -0.14720765, -0.33407348, -0.0394..."
697,4c56ff4ce4aaf9573aa5dff913df997a,2820,0.743583,True,trouble thing freezer nature shop dont glass d...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.00,"[-0.24465466, 0.20265263, -0.079341084, 0.1308..."
116,06409663226af2f3114485aa4e0a23b4,3424,0.000000,True,karma cashew dont sell quickly stock one varie...,False,Snacks,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.25,"[0.112255506, 0.25204742, -0.10517745, -0.6170..."


## Find ideal and worst score

In [19]:
ideal_score = weighted_df[rating_priorities].max(axis=0)
negative_score = weighted_df[rating_priorities].min(axis=0)

In [20]:
random_weights

array([0.70682274, 0.60170271, 0.77066022, 0.96678918, 0.96349508,
       0.27436838, 0.74358268])

In [21]:
ideal_score

rating_packaging    0.706823
rating_price        0.601703
rating_quality      0.770660
rating_service      0.966789
rating_shipping     0.963495
rating_taste        0.274368
avg_rating          0.743583
sentiment_score     1.000000
dtype: float64

In [22]:
negative_score

rating_packaging    0.0
rating_price        0.0
rating_quality      0.0
rating_service      0.0
rating_shipping     0.0
rating_taste        0.0
avg_rating          0.0
sentiment_score     0.0
dtype: float64

In [23]:
# Check if each column in rating_priorities contains any zeros
# zero_counts = (weighted_df[rating_priorities] == 0).sum()
# print("Count of zeros in each column:")
# print(zero_counts)

# # Alternatively, print minimum values directly to confirm
# print("Minimum values in each column:")
# print(weighted_df[rating_priorities].min(axis=0))

In [24]:
# Display rows where any of the columns in rating_priorities have a 0
# rows_with_zeros = weighted_df[(weighted_df[rating_priorities] == 0).any(axis=1)]

# print("Rows with at least one 0 in rating_priorities columns:")
# print(rows_with_zeros)

## Calculate Euclidean Distances

In [25]:
weighted_rating_matrix = weighted_df[rating_priorities]
weighted_rating_matrix

,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,avg_rating,sentiment_score
1502,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.371791,1.00
954,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.371791,0.75
1956,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
1658,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.743583,0.50
1344,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.371791,0.75
719,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.743583,1.00
1643,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.371791,1.00
1,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.743583,1.00
697,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.743583,0.00
116,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.25


In [26]:
distance_to_ideal_solution = np.sqrt((weighted_rating_matrix - ideal_score)**2).sum(axis=1)
distance_to_ideal_solution

1502    2.513711
954     2.763711
1956    6.027421
1658    0.500000
1344    2.763711
719     0.000000
1643    2.513711
1       0.000000
697     1.000000
116     5.777421
dtype: float64

In [27]:
distance_to_worst_solution = np.sqrt((weighted_rating_matrix - negative_score)**2).sum(axis=1)
distance_to_worst_solution

1502    3.513711
954     3.263711
1956    0.000000
1658    5.527421
1344    3.263711
719     6.027421
1643    3.513711
1       6.027421
697     5.027421
116     0.250000
dtype: float64

## Calculate Similarity Score

In [28]:
df_with_similarity_score = weighted_df.copy()

In [29]:
similarity_scores = distance_to_worst_solution / (distance_to_ideal_solution + distance_to_worst_solution)
df_with_similarity_score["similarity_score"] = similarity_scores

In [30]:
df_with_similarity_score.head()

,brand_id,review_id,avg_rating,is_product_recommended,review_content,is_anonymous,display_name,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,sentiment_score,review_embedding,similarity_score
1502,bbcbff5c1f1ded46c25d28119a85c6c2,3152,0.371791,True,popular hence sold people love often buy bulk ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,1.00,"[-0.21371414, 0.06628278, -0.023210816, -0.421...",0.582954
954,69cb3ea317a32c4e6143e665fdb20b14,2782,0.371791,True,people love taste packaging really cool funky ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.75,"[-0.3612504, -0.35247555, -0.15716112, -0.0831...",0.541477
1956,f8c1f23d6a8d8d7904fc0ea8e066b3bb,3171,0.000000,True,think sale werent high enough decided start se...,True,Snacks,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,"[0.031356554, -0.36868742, -0.1400484, -0.2579...",0.000000
1658,c7e1249ffc03eb9ded908c236bd1996d,2962,0.743583,True,selling moving pretty well first got much thin...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.50,"[-0.30708426, -0.73055637, -0.2741928, -0.2398...",0.917046
1344,a01a0380ca3c61428c26a231f0e49a09,1922,0.371791,True,label price fine last 30 day weve sold twelve ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,0.75,"[-0.42296505, 0.0075865746, -0.20753312, 0.198...",0.541477


## Return Top Brands

In [31]:
# Get the top 5 brands
# df_drop_duplicate = df_with_similarity_score.drop_duplicates(subset='brand_id')
top_5_brands = df_with_similarity_score.nlargest(5, 'similarity_score')
# top_5_brands
top_5_brands

,brand_id,review_id,avg_rating,is_product_recommended,review_content,is_anonymous,display_name,rating_packaging,rating_price,rating_quality,rating_service,rating_shipping,rating_taste,sentiment_score,review_embedding,similarity_score
719,4f6ffe13a5d75b2d6a3923922b3922e5,2924,0.743583,True,like sell reorder everything good since still ...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,1.0,"[-0.31779307, -0.18992049, -0.039039858, -0.22...",1.000000
1,00411460f7c92d2124a67ea0f4cb5f85,3718,0.743583,True,carry currently stock selling well much need p...,True,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,1.0,"[-0.5571632, -0.14720765, -0.33407348, -0.0394...",1.000000
1658,c7e1249ffc03eb9ded908c236bd1996d,2962,0.743583,True,selling moving pretty well first got much thin...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.5,"[-0.30708426, -0.73055637, -0.2741928, -0.2398...",0.917046
697,4c56ff4ce4aaf9573aa5dff913df997a,2820,0.743583,True,trouble thing freezer nature shop dont glass d...,False,Snacks,0.706823,0.601703,0.77066,0.966789,0.963495,0.274368,0.0,"[-0.24465466, 0.20265263, -0.079341084, 0.1308...",0.834092
1502,bbcbff5c1f1ded46c25d28119a85c6c2,3152,0.371791,True,popular hence sold people love often buy bulk ...,False,Snacks,0.353411,0.300851,0.38533,0.483395,0.481748,0.137184,1.0,"[-0.21371414, 0.06628278, -0.023210816, -0.421...",0.582954


In [32]:
random_weights

array([0.70682274, 0.60170271, 0.77066022, 0.96678918, 0.96349508,
       0.27436838, 0.74358268])